In [1]:
import pprint

import numpy as np
import pandas as pd
from attrs import asdict

from skrough.algorithms import hooks
from skrough.algorithms.key_names import (
    CONFIG_CANDIDATES_SELECT_RANDOM_MAX_COUNT,
    CONFIG_CHAOS_FUN,
    CONFIG_EPSILON,
    CONFIG_SELECT_ATTRS_CHAOS_SCORE_BASED_MAX_COUNT,
    INPUT_DATA_X,
    INPUT_DATA_Y,
    VALUES_CHAOS_SCORE_APPROX_THRESHOLD,
    VALUES_RESULT_ATTRS,
    VALUES_X,
    VALUES_X_COUNTS,
    VALUES_Y,
    VALUES_Y_COUNT,
)
from skrough.algorithms.meta import describe, processing, stage
from skrough.algorithms.reusables.attrs_greedy import greedy_stage
from skrough.chaos_measures import entropy
from skrough.checks import check_if_approx_reduct
from skrough.dataprep import (
    prepare_factorized_array,
    prepare_factorized_data,
    prepare_factorized_vector,
)
from skrough.structs.attrs_subset import AttrsSubset

In [2]:
df = pd.DataFrame(
    np.array(
        [
            ["sunny", "hot", "high", "weak", "no"],
            ["sunny", "hot", "high", "strong", "no"],
            ["overcast", "hot", "high", "weak", "yes"],
            ["rain", "mild", "high", "weak", "yes"],
            ["rain", "cool", "normal", "weak", "yes"],
            ["rain", "cool", "normal", "strong", "no"],
            ["overcast", "cool", "normal", "strong", "yes"],
            ["sunny", "mild", "high", "weak", "no"],
            ["sunny", "cool", "normal", "weak", "yes"],
            ["rain", "mild", "normal", "weak", "yes"],
            ["sunny", "mild", "normal", "strong", "yes"],
            ["overcast", "mild", "high", "strong", "yes"],
            ["overcast", "hot", "normal", "weak", "yes"],
            ["rain", "mild", "high", "strong", "no"],
        ],
        dtype=object,
    ),
    columns=["Outlook", "Temperature", "Humidity", "Wind", "Play"],
)
TARGET_COLUMN = "Play"
x, x_counts, y, y_count = prepare_factorized_data(df, TARGET_COLUMN)

In [5]:
import skrough.typing as rght
from skrough.algorithms.exceptions import LoopBreak
from skrough.structs.group_index import GroupIndex
from skrough.structs.state import ProcessingState


def xy(
    state: ProcessingState,
) -> None:
    state.values[VALUES_X] = x
    state.values[VALUES_X_COUNTS] = x_counts
    state.values[VALUES_Y] = y
    state.values[VALUES_Y_COUNT] = y_count
    state.values[VALUES_CHAOS_SCORE_APPROX_THRESHOLD] = 0.0001
    state.values[VALUES_RESULT_ATTRS] = [2, 3, 1, 0]


get_reduct = processing.ProcessingMultiStage.from_hooks(
    init_multi_stage_hooks=[
        xy,
    ],
    stages=[],
    finalize_hooks=None,
    prepare_result_fun=hooks.prepare_result_hooks.prepare_result_hook_attrs_subset,
)

import importlib

importlib.reload(hooks)

eps = 0.0
chaos_measure = entropy
result: AttrsSubset = get_reduct(
    input_data={
        # INPUT_DATA_X: x,
        # INPUT_DATA_Y: y,
    },
    config={
        CONFIG_CHAOS_FUN: chaos_measure,
        CONFIG_EPSILON: eps,
    },
)
result

AttrsSubset(attrs=[2, 3, 0])